In [1]:
%cd /projappl/project_2006327/Detectron/datasets/2024

/projappl/project_2006327/Detectron/datasets/2024


In [2]:
from detectron2.utils.logger import setup_logger
logger = setup_logger()

import os
import cv2
import pickle 
import random
import detectron2
import numpy as np
from detectron2.config import get_cfg
from detectron2.config import CfgNode
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader

# Suppress some user warnings
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
# Load your configuration
cfg = get_cfg()

#set directory for the best model
output_dir = "output/faster_rcnn_object_detector_hook"
cfg.OUTPUT_DIR = output_dir

#load the configuration from cfg.pickle
with open ("output/faster_rcnn_object_detector_hook/cfg.pickle", "rb") as f:
    cfg = pickle.load(f)
    cfg = CfgNode(cfg)
    
#load the model weight
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_best.pth")

#create a predictor using the loaded configuration
predictor = DefaultPredictor(cfg)

[02/06 09:22:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output/faster_rcnn_object_detector_hook/model_best.pth ...


### Register datasets

In [4]:
# Some configurations for train, val and test
from detectron2.data.datasets import register_coco_instances

#directory
name_ds = "coco_data"
name_ds_test = name_ds + "_test"
image_root_test = name_ds + "/test"
#annotations
af = "_annotations.coco.json"
json_file_test = name_ds + "/test/" + af

In [5]:
## test dataset
register_coco_instances(
    name = name_ds_test,
    metadata = {},
    json_file = json_file_test,
    image_root = image_root_test
    )

In [6]:
#set the dataset
cfg.DATASETS.TEST = (name_ds_test,)

In [7]:
#Access the test data from the cfg file- pickle file
coco_data_test = cfg.DATASETS.TEST[0]
print(f"Using test dataset: {coco_data_test}")

Using test dataset: coco_data_test


### Inference

In [8]:
# Load your configuration
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_best.pth")  # Load your trained model weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25  # Set a threshold for detections

# Create an evaluator
evaluator = COCOEvaluator("coco_data_test", cfg, False, output_dir="./output")

# Load the test data loader
test_loader = build_detection_test_loader(cfg, "coco_data_test")

# Perform inference on the test dataset
results = inference_on_dataset(predictor.model, test_loader, evaluator)

# Calculate precision and recall from the evaluator
precision = results["bbox"]["AP"]
#recall = results["bbox"]["AR"]

print("Precision:", precision)
#print("Recall:", recall)

WARNING [02/06 09:22:32 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[02/06 09:22:32 d2.data.datasets.coco]: Loaded 277 images in COCO format from coco_data/test/_annotations.coco.json
[02/06 09:22:32 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|     C      | 2642         |
|            |              |
[02/06 09:22:32 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[02/06 09:22:32 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[02/06 09:22:32 d2.data.common]: Serializing 277 elements to byte tensors and concatenating them all ...
[02/06 09:22:32 d2.data.common]: Serialized dataset takes 0.22 MiB
[02/06 09:22:32 d2.evaluation.evaluator]: S